# Implement versions of the naive exact matching and Boyer-Moore algorithms that additionally 
# count and return (a) the number of character comparisons performed and 
# (b) the number of alignments tried. 
# Roughly speaking, these measure how much work the two different algorithms are doing.

In [1]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [2]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [4]:
def naive_count(p, t):
    occurrences = []
    na = 0
    cc = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        na += 1
        match = True
        for j in range(len(p)): # loop over characters
            cc += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record    
    return occurrences, na, cc

In [5]:
# Example 1
p = 'word'
t = 'there would have been a time for such a word'
occurrences, na, cc = naive_count(p, t)
print(occurrences, na, cc)

[40] 41 46


In [6]:
# Example 2
p = 'needle'
t = 'needle need noodle needle'
occurrences, na, cc = naive_count(p, t)
print(occurrences, na, cc)

[0, 19] 20 35


In [7]:
def boyer_moore_count(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    na = 0
    cc = 0
    while i < len(t) - len(p) + 1:
        na += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            cc += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, na, cc

In [11]:
cd "C:\Users\Ramya\Desktop\Ramya\GenomicDataScience\Algorithms for GDS"

C:\Users\Ramya\Desktop\Ramya\GenomicDataScience\Algorithms for GDS


In [12]:
from bm_preproc import BoyerMoore

In [13]:
# Example 1
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, na, cc = boyer_moore_count(p, p_bm, t)
print(occurrences, na, cc)

[40] 12 15


In [14]:
# Example 2
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, na, cc = boyer_moore_count(p, p_bm, t)
print(occurrences, na, cc)

[0, 19] 5 18


In [15]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [165]:
t = readGenome("chr1.GRCh38.excerpt.fasta") # reading the excerpt of human chromosome 1 for further analysis

# Question 1
# How many alignments does the naive exact matching algorithm try when matching the string 
# GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1? 
# (Don't consider reverse complements.)

In [17]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
occurrences, na, cc = naive_count(p, t)
print(occurrences, na, cc)

[56922] 799954 984143


# Question 2
# How many character comparisons does the naive exact matching algorithm try when matching the string 
# GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1? 
# (Don't consider reverse complements.)

In [18]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
occurrences, na, cc = naive_count(p, t)
print(occurrences, na, cc)

[56922] 799954 984143


# Question 3
# How many alignments does Boyer-Moore try when matching the string 
# GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1? 
# (Don't consider reverse complements.)

In [19]:
p_bm = BoyerMoore(p, alphabet = 'ACGT')
occurrences, na, cc = boyer_moore_count(p, p_bm, t)
print(occurrences, na, cc)

[56922] 127974 165191


# Question 4
# Implement the pigeonhole principle using Index to find exact matches for the partitions. 
# Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). 
# We will use an 8-mer index.

# Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, 
# finds all approximate occurrences of P within T with up to 2 mismatches. 
# Insertions and deletions are not allowed. Don't consider any reverse complements.

# How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, 
# occur with up to 2 substitutions in the excerpt of human chromosome 1? (Don't consider reverse complements here.)

# Hint 1: Multiple index hits might direct you to the same match multiple times, 
# but be careful not to count a match more than once.

# Hint 2: You can check your work by comparing the output of your new function to that of the 
# naive_2mm function implemented in the previous module.

In [27]:
import bisect 

class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [144]:
index = Index(t, 8)

In [29]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"

In [33]:
def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:len(p)]:
            offsets.append(i)
    return offsets

In [80]:
def approximate_match_index(p, index, n):
    index = index
    segment_length = int(round(len(p)/(n+1)))
    all_matches = []
   
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = queryIndex(p[start:end], t, index)
        
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
                        
            if mismatches <= n:
                if (m-start) not in all_matches:
                    all_matches.append(m-start)
                           
    return all_matches

In [84]:
match_indexes = approximate_match_index(p, index, 2)
print(match_indexes)
print(len(match_indexes))

[56922, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 160162, 273669, 421221, 429299, 465647, 551134, 724927, 635931, 747359]
19


In [60]:
# Compare with naive exact matching with 2 mismatches allowed

def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = 0
        mm = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mm += 1
                if mm > 2: # allowing upto 2 mismatches
                    break
            else:
                match +=1
        if match >= (len(p)-2) and mm <= 2: # both mismatches and actual matches recorded
            if i not in occurrences:
                occurrences.append(i)
        if match == len(p):
            if i not in occurrences:
                occurrences.append(i)
                
    return occurrences

In [61]:
match = naive_2mm(p, t)
print(match)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


# Question 5
# Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of 
# GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?
# (Don't consider reverse complements.)
# Hint: You should be able to use the boyer_moore function (or the slower naive function) to double-check your answer.

In [88]:
def approximate_match_indexhits(p, index, n):
    index = index
    segment_length = int(round(len(p)/(n+1)))
    all_matches = []
    index_hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = queryIndex(p[start:end], t, index)
        index_hits += len(matches)
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
                        
            if mismatches <= n:
                if (m-start) not in all_matches:
                    all_matches.append(m-start)
                      
    return index_hits, all_matches

In [89]:
all_indexhits, match_indexes = approximate_match_indexhits(p, index, 2)
print(all_indexhits)
print(match_indexes)

90
[56922, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 160162, 273669, 421221, 429299, 465647, 551134, 724927, 635931, 747359]


# Question 6
# Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, 
# finds all approximate occurrences of P within T with up to 2 mismatches.

# When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT 
# with up to 2 substitutions in the excerpt of human chromosome 1? (Again, don't consider reverse complements.)

In [166]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [167]:
subseq_ind = SubseqIndex(t, 8, 3)

In [168]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'

In [169]:
def subseq_queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:len(p)]:
            offsets.append(i)
    return offsets

In [170]:
def approximate_match_subseq_indexhits(p, index, n):
    index = index
    segment_length = int(round(len(p)/(n+1)))
    all_matches = []
    index_hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = subseq_ind.query(p[i:])
        print(matches)
        index_hits += len(matches)
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
                        
            if mismatches <= n:
                if (m-start) not in all_matches:
                    all_matches.append(m-start)
                      
    return index_hits, all_matches

In [171]:
ind_hits, match_indexes = approximate_match_subseq_indexhits(p, subseq_ind, 2)
print(ind_hits)
print(match_indexes)

[56922, 67486, 83863, 84641, 84775, 124024, 147558, 191452, 199607, 262042, 262174, 273669, 322735, 364263, 421354, 454332, 465647, 471966, 472634, 489019, 558456, 579737, 596898, 635931, 651523, 657496, 658702, 681737, 707151, 712449, 717706, 719418, 719557, 746620, 747359]
[22398, 32640, 56923, 84642, 100012, 108111, 137575, 147559, 151719, 160163, 160730, 262043, 273670, 364264, 366819, 421222, 429300, 465648, 479031, 551135, 551828, 635932, 657497, 681738, 717707, 724928, 745641, 783347, 794643]
[23005, 56924, 141048, 160731, 191454, 262044, 349191, 364265, 429301, 657498, 704733, 717708, 724929, 747361, 766421]
79
[56922, 84641, 84775, 147558, 191452, 262042, 273669, 322735, 364263, 465647, 471966, 472634, 635931, 651523, 657496, 681737, 707151, 712449, 717706, 719418, 746620, 747359]
